In [ ]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

load_dotenv(override=True)

DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)

# Choose subontology (CCO, MFO or BPO)
SO = 'MFO'

## Reading fasta, obo and tsv files

In [ ]:
from Bio import SeqIO

sequences = [rec.seq for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]
ids = [rec.id for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]

In [ ]:
print("There are {} sequences in the dataset.".format(len(sequences)))

In [ ]:
import networkx
import obonet

# Read the taxrank ontology
url = os.path.join(DATA_PATH, "Train/go-basic.obo")
graph = obonet.read_obo(url)



In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')

dfSO = df.loc[df["aspect"]==SO]
uniqueTerms = dfSO["term"].unique()
termsArr = list(dfSO["term"].to_numpy())

uniqueTermsDict={}
for i,el in enumerate(uniqueTerms):
    uniqueTermsDict[el] = i
    
print(dfSO.shape)
df=dfSO

Test for the first entry:

In [ ]:
df.set_index("EntryID", inplace=True)
df.head(10)

In [ ]:
testID = df.index.to_list()[0]

In [ ]:
%timeit df.loc[testID]

In [ ]:
df.loc[testID]

In [ ]:
np.unique(df["aspect"].to_numpy())

## GO analysis

In [ ]:
item_counts = df["term"].value_counts()
print(item_counts[0:20])

In [ ]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}
print(id_to_name['GO:0031981'] )

## Label encoding

The task is a multilabel classification: The output has several possible targets (Gene Ontologies) but each can only be 1 (existing) or 0 (non existing)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

topGOs= item_counts
topGOs=topGOs.index.to_list()

mlb = MultiLabelBinarizer()
mlb.fit([topGOs])

dftest=df.loc[testID]
indices = dftest["term"].to_numpy()
print(indices)
print(mlb.transform([indices]))
print(len(mlb.classes_))

with open(os.path.join(DATA_PATH,'MLB_'+SO+'.pkl'), 'wb') as f:
    pickle.dump(mlb, f)

Extract label weights from IA

In [ ]:
dfIa = pd.read_csv(os.path.join(DATA_PATH, "IA.txt"), sep='\t', header=None)

dfIa.set_index(0, inplace=True)

labelWeights=[]
allIndices = dfIa.index.tolist()


for go in topGOs:
    if go in allIndices:
        labelWeights.append(dfIa.loc[go].to_numpy()[0])
    else:
        labelWeights.append(1)

In [ ]:
len(labelWeights)

## Amino acids encoding

In [ ]:
aminos_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X']

In [ ]:
aa_dict = {'A': 1, 'B':24, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 21, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 22, 'V': 18, 'W': 19, 'Y': 20, 'X':30, 'Z':23}

## Build Dataset

In [ ]:
seqLengths = [len(seq) for seq in sequences]
maxLen = max(seqLengths)
print("The max. length of the sequences is {}".format(maxLen))

In [ ]:

dfAll=pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')

soEntries = dfAll.loc[dfAll["aspect"]==SO]
soEntryIds = soEntries["EntryID"].unique()

# print(len(seqEntries))
print(soEntryIds)

SoSequences = []
for entry in soEntryIds:
    SoSequences.append(sequences[ids.index(entry)])

print(len(SoSequences))
dfAll.set_index("EntryID", inplace=True)

In [ ]:
import warnings

TRAIN_VAL_SPLIT = 0.7

#Use numpy vectorize to speed up the mapping (hopefully)
mapping = lambda x: aa_dict[x]
vectMapping = np.vectorize(mapping)

# Shuffle the data
import random
random.seed(516213)
c = list(zip(sequences, ids))
random.shuffle(c)
sequencesShuffle, idsShuffle = zip(*c)

#reduce data for now
# sequencesShuffle = sequencesShuffle[0:10000]
# idsShuffle = idsShuffle[0:10000]

#Train Validation Split
split = int(np.floor(len(sequencesShuffle)*TRAIN_VAL_SPLIT))
print(split)
trainSeq = sequencesShuffle[0:split]
valSeq = sequencesShuffle[split+1:]
trainIds = idsShuffle[0:split]
valIds = idsShuffle[split+1:]


def generator(padding=True):
    for i,seq in enumerate(trainSeq):
        entryId = trainIds[i]
        
        if entryId in soEntryIds:
          labelData = df.loc[entryId]
          # indices = labelData["termToken"].to_numpy()
          indices = labelData["term"].to_numpy()
        else:
          indices=[]

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]

def generatorVal(padding=True):
    for i,seq in enumerate(valSeq):
        entryId = valIds[i]
        if entryId in soEntryIds:
          labelData = df.loc[entryId]
          # indices = labelData["termToken"].to_numpy()
          indices = labelData["term"].to_numpy()
        else:
          indices=[]

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]
        

In [ ]:
g = generator()
test = next(g)
print("The first sample sequence: {}".format(test[0]))
print("The first sample has {} classes".format(np.count_nonzero(test[1])))

## Tensorflow Classification

In [ ]:
import tensorflow as tf


dataset = tf.data.Dataset.from_generator(generator, output_signature=(
         tf.TensorSpec(shape=(maxLen,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))
print(list(dataset.take(1)))

datasetVal = tf.data.Dataset.from_generator(generatorVal, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))

In [ ]:
%load_ext tensorboard


In [ ]:
from tensorflow.keras import layers

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=100

def createModel():
    inputs = tf.keras.Input(shape=(maxLen,))
    x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True, name="embedding")(inputs)
    x=layers.GlobalAveragePooling1D()(x)
    x=layers.Reshape((EMBED_DIM,1))(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    x=layers.Flatten()(x)
    x=layers.Dense(32)(x)
    x=layers.LeakyReLU()(x)
    x=layers.Dense(32)(x)
    x=layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedConvModel")

model = createModel()

model.summary()


In [ ]:

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=100

def createRnnModel():
    inputs = tf.keras.Input(shape=(maxLen,))
    x = tf.keras.layers.Masking(0)(inputs)
    x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(x)

    # x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
    # x = layers.Bidirectional(layers.GRU(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.GRU(16))(x)
    # x = layers.LSTM(32)(x)
    x = layers.Dense(16)(x)
    x = layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedRnnModel")

# model = createRnnModel()

# model.summary()

In [ ]:
import matplotlib.pyplot as plt
#Learning rate schedule
initial_learning_rate = 0.01
decaySteps=5000
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate, first_decay_steps=decaySteps/4,
                                                                t_mul=2.0, m_mul=0.7, alpha=0.001)
# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate, decay_steps=decaySteps, alpha=0.01)
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,decay_steps=decaySteps,decay_rate=0.9,staircase=False)
step = np.linspace(0,decaySteps)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
# plt.yscale("log")
plt.plot(step, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('step')
_ = plt.ylabel('Learning Rate')

In [ ]:
from tensorflow.keras.losses import Loss

class WeightedBinaryCE(Loss):
    # initialize instance attributes
    def __init__(self, classWeights):
        super(WeightedBinaryCE, self).__init__()
        self.classWeights= tf.constant(classWeights, dtype=tf.dtypes.float32)
        
    # Compute loss
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred,tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        term_0 = tf.math.multiply(self.classWeights * tf.math.subtract(1.0 , y_true), tf.math.log(tf.math.subtract(1.0, y_pred) + tf.keras.backend.epsilon()))
        term_1 = tf.math.multiply(self.classWeights * y_true, tf.math.log(y_pred + tf.keras.backend.epsilon()))
        losses = term_0 + term_1
        return -tf.reduce_mean(losses, axis=0)
    

testLoss = WeightedBinaryCE([1,1,0.5])
labelsTrue = tf.constant([[1,0,1], [1,1,0]])
labelsPred = tf.constant([[0.9,0.1,0.9], [0.99,0.1,0.1]])

testLoss(labelsTrue, labelsPred)

In [ ]:


class WeightedF1(tf.keras.metrics.Metric):
    def __init__(self, classWeights, threshold=0.5):
        super(WeightedF1, self).__init__()
        self.classWeights= tf.constant(classWeights, dtype=tf.dtypes.float32)
        self.threshold= tf.constant(threshold, dtype=tf.dtypes.float32)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.total = self.add_weight('total', initializer='zeros')

    def update_state(self, y_true, y_pred):
        m = tf.math.count_nonzero(tf.reduce_min(y_pred, axis=1))
        n = tf.shape(y_pred)[0]
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.math.greater_equal(y_pred,self.threshold)
   

        tp = tf.math.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
        tp = tf.cast(tp, self.dtype)
        tp = tf.math.multiply(tp, self.classWeights)
        tp = tf.cast(tp, self.dtype)

        tn = tf.math.logical_and(tf.equal(y_true, False), tf.equal(y_pred, False))
        tn = tf.cast(tn, self.dtype)
        tn = tf.math.multiply(tn, self.classWeights)
        tn = tf.cast(tn, self.dtype)

        fp = tf.math.logical_and(tf.equal(y_true, True), tf.equal(y_pred, False))
        fp = tf.cast(fp, self.dtype)
        fp = tf.math.multiply(fp, self.classWeights)
        fp = tf.cast(fp, self.dtype)

        fn = tf.math.logical_and(tf.equal(y_true, False), tf.equal(y_pred, True))
        fn = tf.cast(fn, self.dtype)
        fn = tf.math.multiply(fn, self.classWeights)
        fn = tf.cast(fn, self.dtype)

        pr = tf.math.divide(tf.math.reduce_sum(tp, axis=1),tf.math.reduce_sum(tp+fp+tf.keras.backend.epsilon(), axis=1))
        m = tf.cast(m, pr.dtype)
        if(m>0):
            wPr = tf.math.reduce_sum(pr)/m
        else:
            wPr = tf.constant(0, tf.dtypes.float32)

        re = tf.math.divide(tf.math.reduce_sum(tp, axis=1),tf.math.reduce_sum(tp+fn+tf.keras.backend.epsilon(), axis=1))
        n = tf.cast(n, re.dtype)
        wRe = tf.math.reduce_sum(re)/n

        res = tf.math.divide(2*wPr*wRe, wPr + wRe + tf.keras.backend.epsilon())

        self.f1.assign_add(res)
        self.total.assign_add(1)

    def result(self):
        return tf.math.divide(self.f1, self.total)

testM = WeightedF1(classWeights=[1,1,2])
labelsTrue = tf.constant([[1,0,1], [1,1,0]])
labelsPred = tf.constant([[0.8,0.1,0.9], [0.9,0.9,0.6]])

testM.update_state(labelsTrue, labelsPred)
testM.result()

In [ ]:
import tensorflow_addons as tfa

BATCH_SIZE=32
LOG_INTERVAL=10
epochs = 10
saveModel=False


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

log_dir = "./logs/"+model.name+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_"+SO
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_graph=True, update_freq=5)


summary_writer = tf.summary.create_file_writer(log_dir)

# Instantiate an optimizer .
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
optimizer = tf.keras.optimizers.AdamW(learning_rate=lr_schedule)

# Instantiate a loss function.
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = WeightedBinaryCE(labelWeights)

train_acc_metric = tf.keras.metrics.BinaryAccuracy()
train_f1_metric = WeightedF1(classWeights=labelWeights, threshold=0.5)
# train_f1_metric = tfa.metrics.F1Score(num_classes=len(mlb.classes_), threshold=0.5, average="macro")
val_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_f1_metric = WeightedF1(classWeights=labelWeights, threshold=0.5)
# val_f1_metric = tfa.metrics.F1Score(num_classes=len(mlb.classes_), threshold=0.5, average="macro")

batchedDataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
batchedDatasetVal = datasetVal.batch(BATCH_SIZE, drop_remainder=False)

batchedDataset = batchedDataset.cache(os.path.join(DATA_PATH, "datasetCache"+SO))
batchedDatasetVal = batchedDatasetVal.cache(os.path.join(DATA_PATH, "datasetCacheVal"+SO))

@tf.function()
def trainStep(x_batch_train, y_batch_train):
    # Open a GradientTape to record the operations run
    # during the forward pass, which enables auto-differentiation.
    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        probs = model(x_batch_train, training=True) 

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y_batch_train, probs)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)
    #Gradient clipping
    # grads = [tf.clip_by_norm(g, 2.0) for g in grads]

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y_batch_train, probs)
    train_f1_metric.update_state(y_batch_train, probs)
    return loss_value, grads

maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(batchedDataset):

        loss_value, grads=trainStep(x_batch_train,y_batch_train)

        # Log 
        if step % LOG_INTERVAL == 0:
            template = 'Epoch {}/Step {}, Loss: {:.5f}, Accuracy: {:.5f}, F1: {:.4f}, lr: {:.5f}'
            print (template.format(epoch+1, step,loss_value, 
                                    train_acc_metric.result(),train_f1_metric.result(), optimizer.learning_rate.numpy()))
            # print([tf.norm(grad, ord=2).numpy() for grad in grads])
            with summary_writer.as_default():
                tf.summary.scalar('loss', loss_value, step=maxStep*epoch+step)
                tf.summary.scalar('accuracy', train_acc_metric.result(), step=maxStep*epoch+step)
                tf.summary.scalar('f1', train_f1_metric.result(), step=maxStep*epoch+step)
                tf.summary.scalar('learning rate', optimizer.learning_rate.numpy(), step=maxStep*epoch+step)
                summary_writer.flush()

    
    train_acc_metric.reset_states()
    train_f1_metric.reset_states()

    maxStep=step

    print("Epoch finished. Start validation")
    for x_batch_val, y_batch_val in batchedDatasetVal:
        valProbs = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, valProbs)
        val_f1_metric.update_state(y_batch_val, valProbs)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    val_f1 = val_f1_metric.result()
    val_f1_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Validation f1: %.4f" % (float(val_f1),))
    with summary_writer.as_default():
        tf.summary.scalar('valAcc', float(val_acc), step=epoch)
        tf.summary.scalar('valF1', float(val_f1), step=epoch)
        summary_writer.flush()
    if saveModel:
      model.save(os.path.join(DATA_PATH, "model_"+SO+"_epoch_{}_valAcc{:.3f}".format(epoch, float(val_acc))))

In [ ]:
# for layer in model.layers: print(layer.get_config(), layer.get_weights())

In [ ]:

probs= model.predict(tf.expand_dims(list(datasetVal.take(32))[10][0], 0))
prediction= [1 if p > 0.5 else 0 for p in probs[0]]
probabilities= probs[probs>0.5]
# classes = np.argwhere(prediction)
print(mlb.inverse_transform(np.array([prediction])))
print(probabilities)